# Data Extraction


In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import concurrent.futures

import pandas as pd
from tqdm import tqdm

from util import clone_or_pull_repo, export_commit_to_csv

## 1. Clone or Pull GitHub Repositories
- reads from `/data/repos.txt`

In [26]:
repo_sources = []

with open("data/repos.txt") as f:
    for line in f:
        repo_sources.append(line.strip())

repos = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    repos = executor.map(clone_or_pull_repo, repo_sources)
repos = list(repos)

Pulling openssh-portablePulling openssl

Pulling httpd
Pulling nginx
[DONE] Pulling openssl
[DONE] Pulling nginx
[DONE] Pulling openssh-portable
[DONE] Pulling httpd


## 2. Extraction

### To CSV

In [27]:
# count commits in all repos using list comprehension
commit_count = sum(1 for repo in repos for commit in repo.iter_commits())

with tqdm(total=commit_count, desc="Exporting commits to CSV") as pbar:
    for repo in repos:
        for commit in repo.iter_commits():
            export_commit_to_csv(commit, "data/commits.csv")
            pbar.update()

Exporting commits to CSV: 100%|██████████| 87071/87071 [13:51<00:00, 104.67it/s]


In [28]:
df = pd.read_csv("data/commits.csv", names=["message", "sha", "remote_url"])
df.head()
df.to_csv("data/commits.csv", index=False)